# EigenFaces with Normalized Cross Correlation (NCC) data

In [ ]:
import numpy as np
from nilearn import image
from nilearn.image import get_data
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from arch.bootstrap import IIDBootstrap
from scipy.stats import kurtosis
import re
import time
import json
import glob
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, classification_report,f1_score,roc_auc_score,recall_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_curve,auc

In [ ]:
path= '/dicom/'
label_path='with_category.csv'
indicative="000"
df_labels=pd.read_csv(label_path,sep=';')
df_labels['id1']=indicative+df_labels['id'].astype(str)
df_labels['path']=path+df_labels['id1']+'/'

In [ ]:
# skipping studies with previous surgeries or artifacts
skip = ['00020023096','00020029351','00030026189','00030037155','00030037273','00050009584','00050002822',
        '00050004047','00050004076','00050004345','00050004468','00050004755']
df_labels=df_labels[~df_labels.id1.isin(skip)]

# 1. Get images and collapsing data

In [ ]:
path_health=df_labels[df_labels.label==1].path
path_pathologic=df_labels[df_labels.label==0].path
space = 'hemispheres'

if(space=='hemispheres'):
    nx=45;ny=109;nz=91
elif(space=='all'):
    nx=91;ny=109;nz=91

dat_healt=getDataFromDirectory_NCC_eigen(path_health.values,space,nx,ny,nz,'mean')
dat_pathologic=getDataFromDirectory_NCC_eigen(path_pathologic.values,space,nx,ny,nz,'mean')

In [ ]:
print('health neuroimaging: ',len(dat_healt['right']))
print('pathologic neuroimaging: ',len(dat_pathologic['right']))

# Eigenfunction by hemispheres

In [ ]:
heat_cov_right, heat_cov_left = imagCov_eigen(dat_healt,'hemispheres')
path_cov_right, path_cov_left = imagCov_eigen(dat_pathologic,'hemispheres')

In [ ]:
heat_cov_left.shape

In [ ]:
# Valores singulares
_,s_h_right,_ = np.linalg.svd(heat_cov_right)
_,s_h_left,_ = np.linalg.svd(heat_cov_left)
_,s_p_right,_ = np.linalg.svd(path_cov_right)
_,s_p_left,_ = np.linalg.svd(path_cov_left)

In [ ]:
comp_healt_rigth = CompNum(s_h_right,0.99)
comp_healt_left = CompNum(s_h_left,0.99)
comp_patho_right = CompNum(s_p_right,0.99)
comp_patho_left = CompNum(s_p_left,0.99)

print("Healthy Right Components:", comp_healt_rigth,
      "\nHealthy Right Components:", comp_healt_left,
     "\nPathology Right Components:", comp_patho_right,
     "\nPathologic Right Components:", comp_patho_left)

In [ ]:
n_comp_healt = [comp_healt_rigth,comp_healt_left]
heat_pca_right, heat_pca_left = imagPCA_eigen(dat_healt,n_comp_healt,'hemispheres')

In [ ]:
heat_pca_right.components_.shape

In [ ]:
heat_pca_left.components_.shape

In [ ]:
n_comp_patho = [comp_patho_right,comp_patho_left]
patho_pca_right, patho_pca_left = imagPCA_eigen(dat_pathologic,n_comp_patho)

In [ ]:
patho_pca_right.components_.shape

In [ ]:
patho_pca_left.components_.shape

## Data projection

In [ ]:
healt_right_projected = heat_pca_right.transform(dat_healt['right'])
healt_left_projected = heat_pca_left.transform(dat_healt['left'])
patho_right_projected = patho_pca_right.transform(dat_pathologic['right'])
patho_left_projected = patho_pca_left.transform(dat_pathologic['left'])

In [ ]:
print("Healthy Right Projected Shape:", healt_right_projected.shape,
      "\nHealthy Left Projected Shape:", healt_left_projected.shape,
     "\nPathology Right Projected Shape:", patho_right_projected.shape,
     "\nPathologic Left Projected Shape:", patho_left_projected.shape)

## Hemispheres comparison

In [ ]:
_ ,_, diff_healthy = hemisCOmpar(healt_right_projected,healt_left_projected,2)
_ ,_, diff_pathology = hemisCOmpar(patho_right_projected,patho_left_projected,2)

In [ ]:
np.quantile(diff_healthy,[0.025,0.05,0.25,0.5,0.75,0.95,0.975])

In [ ]:
np.quantile(diff_pathology,[0.025,0.05,0.25,0.5,0.75,0.95,0.975])

In [ ]:
# plot of 2 variables
p1=sns.kdeplot(diff_healthy, shade=True, color="r")
p1=sns.kdeplot(diff_pathology, shade=True, color="b")

# Eigenfunction all

In [ ]:
path_health=df_labels[df_labels.label==1].path
path_pathologic=df_labels[df_labels.label==0].path
space = 'all'

if(space=='hemispheres'):
    nx=45;ny=109;nz=91
elif(space=='all'):
    nx=91;ny=109;nz=91

dat_healt=getDataFromDirectory_NCC_eigen(path_health.values,space,nx,ny,nz,'mean')
dat_pathologic=getDataFromDirectory_NCC_eigen(path_pathologic.values,space,nx,ny,nz,'mean')

In [ ]:
print('health neuroimaging: ',len(dat_healt['data']))
print('pathologic neuroimaging: ',len(dat_pathologic['data']))

In [ ]:
heat_cov = imagCov_eigen(dat_healt,space='all')
path_cov = imagCov_eigen(dat_pathologic,space='all')

In [ ]:
# Valores singulares
_,s_h,_ = np.linalg.svd(heat_cov)
_,s_p,_ = np.linalg.svd(path_cov)

In [ ]:
comp_healt = CompNum(s_h,0.99)
comp_patho = CompNum(s_p,0.99)

print("Healthy Components:", comp_healt,
     "\nPathology Components:", comp_patho)

In [ ]:
# selelct the greather componentes numbers
n_comp = max(comp_healt,comp_patho)
n_comp = [n_comp]
n_comp

In [ ]:
heat_pca = imagPCA_eigen(dat_healt,n_comp,'all')

In [ ]:
heat_pca.components_.shape

In [ ]:
patho_pca = imagPCA_eigen(dat_pathologic,n_comp,'all')

In [ ]:
patho_pca.components_.shape

# Data projection

### Projection pathologic on helthy space

In [ ]:
healtONhealt = heat_pca.transform(dat_healt['data'])
patholONhealt = heat_pca.transform(dat_pathologic['data'])

In [ ]:
print("Healthy Projected Shape:", healtONhealt.shape,
     "\nPathology Projected Shape:", patholONhealt.shape)

### Projection healthy on pathologic space

In [ ]:
patholONpathol = patho_pca.transform(dat_pathologic['data'])
healtONpathol = patho_pca.transform(dat_healt['data'])

In [ ]:
print("Healthy Projected Shape:", patholONpathol.shape,
     "\nPathology Projected Shape:", healtONpathol.shape)

## Models 1 : with pathologic on helathy space

In [ ]:
h = np.zeros(healtONhealt.shape[0])
p = np.ones(patholONhealt.shape[0])
x = np.concatenate((healtONhealt,patholONhealt),axis=0)
y = np.concatenate((h,p),axis=0)

df = pd.DataFrame(x)
df['y'] = y

In [ ]:
df.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y,random_state=0)

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Comparison Models and cross validation

In [ ]:
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('RF', RandomForestClassifier(n_estimators = 100, random_state = 42)))
models.append(('AGB', AdaBoostClassifier(n_estimators=100, random_state=42)))
models.append(('SGB', GradientBoostingClassifier(n_estimators=100, random_state=42)))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    kfold = KFold(n_splits=10, random_state=2020)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring) 
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()) 
    print(msg)

In [ ]:
fig = plt.figure(figsize=(10, 5)) 
#fig.suptitle('Algorithm Comparison') 
ax = fig.add_subplot(111) 
plt.boxplot(results) 
ax.set_xticklabels(names)
ax.set(xlabel='Model', ylabel='Accuracy')
plt.show()

### Logistics

In [ ]:
classifier = LogisticRegression(solver='newton-cg')
classifier.fit(X_train, y_train)

In [ ]:
y_test_pred = classifier.predict(X_test)

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_test_pred, target_names=["Helthy", "Parhologic"]))

In [ ]:
plt.figure()
plt.title("Heatmap")
classes_dict = {'Actual': y_test.tolist(), 'Predicted': y_test_pred.tolist()}
classes_df = pd.DataFrame(classes_dict, columns=["Actual", "Predicted"])
conf_matrix = pd.crosstab(classes_df['Actual'], classes_df['Predicted'], rownames=['Actual'], colnames=['Predicted'])
ax=sns.heatmap(conf_matrix, annot=True,cmap='Blues', fmt='.0f');
ax.invert_yaxis()
ax.invert_xaxis()

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 100 decision trees
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train)

In [ ]:
y_test_pred = rf.predict(X_test)

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_test_pred, target_names=["Helthy", "Parhologic"]))

In [ ]:
plt.figure()
plt.title("Heatmap")
classes_dict = {'Actual': y_test.tolist(), 'Predicted': y_test_pred.tolist()}
classes_df = pd.DataFrame(classes_dict, columns=["Actual", "Predicted"])
conf_matrix = pd.crosstab(classes_df['Actual'], classes_df['Predicted'], rownames=['Actual'], colnames=['Predicted'])
ax=sns.heatmap(conf_matrix, annot=True,cmap='Blues', fmt='.0f');
ax.invert_yaxis()
ax.invert_xaxis()

### Suport Vectro Machine

In [ ]:
svm = SVC()
svm.fit(X_train, y_train)

In [ ]:
y_test_pred = svm.predict(X_test)

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_test_pred, target_names=["Helthy", "Parhologic"]))

In [ ]:
plt.figure()
plt.title("Heatmap")
classes_dict = {'Actual': y_test.tolist(), 'Predicted': y_test_pred.tolist()}
classes_df = pd.DataFrame(classes_dict, columns=["Actual", "Predicted"])
conf_matrix = pd.crosstab(classes_df['Actual'], classes_df['Predicted'], rownames=['Actual'], colnames=['Predicted'])
ax=sns.heatmap(conf_matrix, annot=True,cmap='Blues', fmt='.0f');
ax.invert_yaxis()
ax.invert_xaxis()

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_test_pred)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure(1,figsize=(10, 5))
#plt.xlim(0, 0.2)
#plt.ylim(0.8, 1)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.plot(fpr, tpr, color='darkorange',
         lw=2,label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

### Gradient Boosting

In [ ]:
GB = AdaBoostClassifier(n_estimators=100, random_state=42)
GB.fit(X_train, y_train)

In [ ]:
y_test_pred = GB.predict(X_test)

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_test_pred, target_names=["Helthy", "Parhologic"]))

In [ ]:
plt.figure()
plt.title("Heatmap")
classes_dict = {'Actual': y_test.tolist(), 'Predicted': y_test_pred.tolist()}
classes_df = pd.DataFrame(classes_dict, columns=["Actual", "Predicted"])
conf_matrix = pd.crosstab(classes_df['Actual'], classes_df['Predicted'], rownames=['Actual'], colnames=['Predicted'])
ax=sns.heatmap(conf_matrix, annot=True,cmap='Blues', fmt='.0f');
ax.invert_yaxis()
ax.invert_xaxis()

### Stocastic Gradient Boosting

In [ ]:
SGB = GradientBoostingClassifier(n_estimators=100, random_state=42)
SGB.fit(X_train, y_train)

In [ ]:
y_test_pred = SGB.predict(X_test)

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_test_pred, target_names=["Helthy", "Parhologic"]))

In [ ]:
plt.figure()
plt.title("Heatmap")
classes_dict = {'Actual': y_test.tolist(), 'Predicted': y_test_pred.tolist()}
classes_df = pd.DataFrame(classes_dict, columns=["Actual", "Predicted"])
conf_matrix = pd.crosstab(classes_df['Actual'], classes_df['Predicted'], rownames=['Actual'], colnames=['Predicted'])
ax=sns.heatmap(conf_matrix, annot=True,cmap='Blues', fmt='.0f');
ax.invert_yaxis()
ax.invert_xaxis()

## Voting Ensemble (SVM and SGB)

In [ ]:
kfold = KFold(n_splits=10, random_state=2020)
# create the sub models
estimators = []
model1 = SVC()
estimators.append(('SVC', model1))
model2 = GradientBoostingClassifier(n_estimators=100, random_state=42)
estimators.append(('SGB', model2))

# create the ensemble model
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble, X_train, y_train, cv=kfold)
print(results.mean())

## Models 2 : with healthy on pathologic space

In [ ]:
h1 = np.zeros(healtONhealt.shape[0])
p1 = np.ones(patholONhealt.shape[0])
x1 = np.concatenate((patholONpathol,healtONpathol),axis=0)
y1 = np.concatenate((h,p),axis=0)

df1 = pd.DataFrame(x1)
df1['y'] = y1

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.3, stratify=y1,random_state=0)

In [ ]:
scaler1 = StandardScaler().fit(X_train1)
X_train1 = scaler1.transform(X_train1)
X_test1 = scaler1.transform(X_test1)

In [ ]:
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('RF', RandomForestClassifier(n_estimators = 100, random_state = 42)))
models.append(('AGB', AdaBoostClassifier(n_estimators=100, random_state=42)))
models.append(('SGB', GradientBoostingClassifier(n_estimators=100, random_state=42)))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    kfold = KFold(n_splits=10, random_state=2020)
    cv_results = cross_val_score(model, X_train1, y_train1, cv=kfold, scoring=scoring) 
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()) 
    print(msg)

In [ ]:
# boxplot algorithm comparison
fig = plt.figure() 
fig.suptitle('Algorithm Comparison') 
ax = fig.add_subplot(111) 
plt.boxplot(results) 
ax.set_xticklabels(names) 
plt.show()

**conclusion:** The projection on pathologic space is worse by model prediction

# Distribution of projection on helthy space

In [ ]:
def normDataProjected(data,order):
    norm = np.linalg.norm(data,axis=1,ord=order)
    return(norm)

In [ ]:
norm_L1_heathy = normDataProjected(healtONhealt,order=1)
norm_L1_pathol = normDataProjected(patholONhealt,order=1)

norm_L2_heathy = normDataProjected(healtONhealt,order=2)
norm_L2_pathol = normDataProjected(patholONhealt,order=2)

In [ ]:
sns.kdeplot(norm_L1_heathy, shade=True, color="r")
sns.kdeplot(norm_L1_pathol, shade=True, color="b")

In [ ]:
sns.kdeplot(norm_L2_heathy, shade=True, color="r")
sns.kdeplot(norm_L2_pathol, shade=True, color="b")

**conclusion:** The distance of helthy images to helthy space is less that pathologic distance

# Save best model

In [ ]:
from sklearn.externals.joblib import dump
from sklearn.externals.joblib import load

# save the model to disk
filename = 'final_models/bestModel_SVM_NCC.sav' 
dump(svm, filename)

In [ ]:
# Save healty
dumped_healty = json.dumps(dat_healt['center_image'], cls=NumpyEncoder) 
dumped_pathologic = json.dumps(dat_pathologic['center_image'], cls=NumpyEncoder)

with open('centerImage_healty.json', 'w') as f:
    json.dump(dumped_healty, f) # , sort_keys=False, indent=4
    
with open('centerImage_pathologic.json', 'w') as f:
    json.dump(dumped_pathologic, f) # , sort_keys=False, indent=4

In [ ]:
import pickle as pk
pk.dump(heat_pca, open("Eigenfaces_healty.pkl","wb"))

# Load Components and Model

In [ ]:
pca_reload = pk.load(open("Eigenfaces_healty.pkl",'rb'))

In [ ]:
pca_reload.components_.shape

In [ ]:
# load the model from disk
loaded_model = load(filename)
result = loaded_model.score(X_test, y_test) 
print(result)